In [1]:
from taxcrunch import multi_cruncher as mc
import os
import pandas as pd
import math

In [2]:
def run_model(folder):
    CURRENT_PATH = os.path.abspath(os.path.dirname('file'))
    path = os.path.join(CURRENT_PATH, folder + "/input_WZ_2026.csv")

    escalate = True
    counter = 1
    while escalate:
        test_df = pd.read_csv(path, header=None)
        test_df.loc[1] = test_df.loc[0]
        test_df.at[1, 0] = 2
        test_df.at[1, 9] = test_df.at[0, 9] + counter
        path_a = os.path.join(CURRENT_PATH, folder + "/input_WZ_2026_a.csv")
        test_df.to_csv(path_a, header=None, index=None)
    
        batch = mc.Batch(path_a)
    
        reform_path = os.path.join(CURRENT_PATH, "TCJA_ext.json")
        output_df = batch.create_table(reform_file=reform_path)
        output_df.to_csv(folder + '/output_WZ_2026.csv')
    
        diff_df = batch.create_diff_table(reform_file=reform_path)
        diff_df.to_csv(folder + '/diff_WZ_2026.csv')
    
        post_tax_income_ext = (output_df['Wages'] - output_df['Individual Income Tax'])[0]
        post_tax_income_exp = post_tax_income_ext + diff_df['Individual Income Tax Diff'][0]
        equiv = (output_df['Wages'] - output_df['Individual Income Tax']+ diff_df['Individual Income Tax Diff'])[1]
    
        if post_tax_income_ext - equiv > 1:
            counter = counter + math.floor(post_tax_income_ext - equiv)
        else:
            escalate = False

    tax_extended = output_df['Individual Income Tax'][0]
    print(tax_extended)
    tax_expired = output_df['Individual Income Tax'][0] - diff_df['Individual Income Tax Diff'][0]
    print(tax_expired)
    
    rate_extended = output_df['Income Tax MTR'][0]
    print(rate_extended)
    rate_expired = output_df['Income Tax MTR'][0] - diff_df['Income Tax MTR Diff'][0]
    print(rate_expired)
    
    needed_extended = test_df[9][0] + test_df[10][0]
    print(needed_extended)
    needed_expired = test_df[9][1] + test_df[10][1]
    print(needed_expired)

In [10]:
run_model('case2')

23419.1636
25228.51
0.240000000121654
0.28000000020256266
160000
162512


In [9]:
run_model('case1')

15577.1306
18521.85
0.21999999989930075
0.24999999986903276
110000
113925


In [11]:
run_model('case3')

27154.265799999994
34393.6
0.22000000026309863
0.25000000023283064
220000
229652


In [12]:
run_model('case4')

33889.7878
38148.1
0.23999999975785613
0.25000000023283064
250000
255783
